In [2]:
import pandas as pd

df = pd.read_csv('Data/trump_tweets_reasons.csv')

In [ ]:
from langchain.llms import Together
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.agents import Tool, initialize_agent, AgentType
from tqdm import tqdm
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize Together AI LLM
llm = Together(
    model="meta-llama/Llama-3-70b-chat-hf",
    together_api_key=os.getenv('TOGETHER_API_KEY'),
    temperature=0.1,
    max_tokens=150
)

def coding_tool(input_str):
    """Tool to classify text based on codebook definition."""
    try:
        import json
        input_dict = json.loads(input_str)
        
        text = input_dict.get('text', '')
        code_name = input_dict.get('code_name', '')
        code_definition = input_dict.get('code_definition', '')
        
        prompt = f"""[INST] Analyze this tweet based on the code definition and provide a Yes or No answer.

Tweet: {text}

Code: {code_name}
Definition: {code_definition}

Return ONLY Yes or No (without any explanation). [/INST]"""

        response = llm.predict(prompt)
        # Clean up response to be exactly Yes or No
        response = response.strip().lower()
        if 'yes' in response:
            return 'Yes'
        elif 'no' in response:
            return 'No'
        return response

    except Exception as e:
        return f"Error: {str(e)}"

# Create the tool
tools = [
    Tool(
        name="TweetCodingTool",
        func=coding_tool,
        description="Analyzes a tweet using a code definition and returns exactly 'Yes' or 'No'."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=2
)

def process_all_tweets(df, codebook):
    """Process all tweets in the DataFrame."""
    df_processed = df.copy()
    df_processed['new_model_code'] = None
    
    # Process all rows with a progress bar
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Processing tweets"):
        try:
            code_definition = codebook.get(row['code_name'], "No definition found.")
            
            input_dict = {
                "text": row['text'],
                "code_name": row['code_name'],
                "code_definition": code_definition
            }
            
            input_str = f"""Use the TweetCodingTool to analyze this tweet.
            Tool input: {json.dumps(input_dict)}"""
            
            response = agent.run(input_str)
            
            # Extract just the Yes/No from the response
            if 'Final Answer:' in response:
                response = response.split('Final Answer:')[-1].strip()
            
            response = response.strip().lower()
            if 'yes' in response:
                df_processed.at[idx, 'new_model_code'] = 'Yes'
            elif 'no' in response:
                df_processed.at[idx, 'new_model_code'] = 'No'
            
            # Save progress every 100 rows
            if idx % 100 == 0 and idx > 0:
                df_processed.to_csv('tweet_analysis_checkpoint.csv', index=False)
                print(f"\nCheckpoint saved at row {idx}")
                
        except Exception as e:
            print(f"\nError processing row {idx}: {str(e)}")
            df_processed.at[idx, 'new_model_code'] = None

    return df_processed

# Your codebook
codebook = {
    'HSTG': "Are hashtags used in this tweet? Exclude hyperlinks from this decision.",
    'ATSN': "Are at signs (\"@\") used in this tweet? Include \"@\" that are present in retweets. Exclude hyperlinks from this decision.",
    'CRIT': "Does Donald Trump criticize another person or idea in this tweet? If the author suggests at any point in the tweet that some person or entity did something wrong, code 'Yes'.",
    'MEDI': "Does Donald Trump make derogatory or condescending statements about the news media in this tweet?",
    'FAMY': "Does Donald Trump reference members of his immediate family in this tweet?",
    'PLCE': "Does this tweet reference the police?",
    'MAGA': "Does this tweet reference Donald Trump's 2016 campaign slogan?",
    'CAPT': "Are there words that contain only capital letters in this tweet?",
    'INDV': "Does this tweet reference an individual person? Exclude self-references to Donald Trump.",
    'MARG': "Does the tweet explicitly reference a marginalized group or groups?",
    'INTN': "Does this tweet reference international topics outside of USA?",
    'PRTY': "Does this tweet reference US political parties?",
    'IMMG': "Does this tweet reference immigration in the US?"
}

if __name__ == "__main__":
    # Process all tweets
    df_results = process_all_tweets(df, codebook)

In [19]:
    # Save final results
df_results.to_csv('tweet_analysis_final.csv', index=False)
print("\nAnalysis complete. Results saved to 'tweet_analysis_final.csv'")


Analysis complete. Results saved to 'tweet_analysis_final.csv'


In [20]:
df_results.head()

,text_id,code_id,code_name,text,original_code,replicated_code,model_code,reason,new_model_code
0,1,1,HSTG,A great guy (with great ratings)! https://t.co...,No,No,No,"There are no hashtags used in this tweet, only...",No
1,1,2,ATSN,A great guy (with great ratings)! https://t.co...,No,No,No,There are no at signs (@) present in this twee...,No
2,1,3,CRIT,A great guy (with great ratings)! https://t.co...,No,No,No,The tweet does not criticize another person or...,No
3,1,4,MEDI,A great guy (with great ratings)! https://t.co...,No,No,No,This tweet does not contain any derogatory or ...,No
4,1,5,FAMY,A great guy (with great ratings)! https://t.co...,No,No,No,The tweet does not reference any members of Do...,No


In [26]:
from langchain.llms import Together
from langchain.agents import Tool, initialize_agent, AgentType
from tqdm import tqdm
import pandas as pd
import os
import time
import json
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def prepare_prompt(text, code_name, code_definition):
    """Prepare prompt for a specific code and text."""
    return f"""You are a qualitative coder who is annotating tweets from Donald Trump's Twitter feed.
    To code this tweet, do the following:
    - First, read the codebook and the tweet.
    - Next, decide which code is most applicable and explain your reasoning for the coding decision.
    - Finally, print the most applicable code and your reason for the coding decision.

    Codebook definition for {code_name}:
    {code_definition}

    Tweet:
    {text}

    Respond with only Yes or No, followed by a colon and your reason."""

# Initialize Together AI LLM
llm = Together(
    model="meta-llama/Llama-3-70b-chat-hf",
    together_api_key=os.getenv('TOGETHER_API_KEY'),
    temperature=0.1,
    max_tokens=100
)

def coding_tool(input_str):
    """Tool to classify text based on codebook definition."""
    try:
        input_dict = json.loads(input_str)
        
        text = input_dict.get('text', '')
        code_name = input_dict.get('code_name', '')
        code_definition = input_dict.get('code_definition', '')
        
        prompt = prepare_prompt(text, code_name, code_definition)
        
        # Add retry logic
        max_retries = 3
        for attempt in range(max_retries):
            try:
                response = llm.predict(prompt)
                response = response.strip()
                
                # Split into decision and reason
                parts = response.split(':', 1)
                if len(parts) == 2:
                    decision, reason = parts[0].strip(), parts[1].strip()
                    if decision.lower() in ['yes', 'no']:
                        return f"{decision}:{reason}"
                
                if attempt < max_retries - 1:
                    time.sleep(1 * (attempt + 1))
                    continue
                    
            except Exception as e:
                if attempt < max_retries - 1:
                    time.sleep(1 * (attempt + 1))
                    continue
                return f"Error:{str(e)}"
                
        return "Error:Invalid response format"

    except Exception as e:
        return f"Error:{str(e)}"

# Create the tool
tools = [
    Tool(
        name="TweetCodingTool",
        func=coding_tool,
        description="Analyzes a tweet using a code definition from the codebook."
    )
]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=1
)

# Define the codebook
codebook = {
    'HSTG': "Are hashtags used in this tweet? Exclude hyperlinks from this decision.",
    'ATSN': "Are at signs (\"@\") used in this tweet? Include \"@\" that are present in retweets. Exclude hyperlinks from this decision.",
    'CRIT': "Does Donald Trump criticize another person or idea in this tweet? If the author suggests at any point in the tweet that some person or entity did something wrong, code 'Yes'.",
    'MEDI': "Does Donald Trump make derogatory or condescending statements about the news media in this tweet?",
    'FAMY': "Does Donald Trump reference members of his immediate family in this tweet?",
    'PLCE': "Does this tweet reference the police?",
    'MAGA': "Does this tweet reference Donald Trump's 2016 campaign slogan?",
    'CAPT': "Are there words that contain only capital letters in this tweet?",
    'INDV': "Does this tweet reference an individual person? Exclude self-references to Donald Trump.",
    'MARG': "Does the tweet explicitly reference a marginalized group or groups?",
    'INTN': "Does this tweet reference international topics outside of USA?",
    'PRTY': "Does this tweet reference US political parties?",
    'IMMG': "Does this tweet reference immigration in the US?"
}

def process_tweets_sample(df, n_rows=100):
    """Process first n_rows tweets and add model predictions and reasons."""
    # Make a copy of the full dataframe
    df_processed = df.copy()
    
    # Rename existing model_code column if it exists
    if 'model_code' in df_processed.columns:
        df_processed = df_processed.rename(columns={
            'model_code': 'original_model_code',
            'reason': 'original_reason'
        })
    
    # Add new columns for the new model's predictions
    df_processed['new_model_code'] = None
    df_processed['new_model_reason'] = None
    
    # Process first n_rows tweet-code combinations
    for idx, row in tqdm(df_processed.iloc[:n_rows].iterrows(), total=n_rows):
        try:
            # Prepare input for the agent
            input_dict = {
                "text": row['text'],
                "code_name": row['code_name'],
                "code_definition": codebook[row['code_name']]
            }
            
            input_str = f"""Use the TweetCodingTool to analyze this tweet.
            Tool input: {json.dumps(input_dict)}"""
            
            # Get agent response
            response = agent.run(input_str)
            
            # Extract decision and reason
            if 'Final Answer:' in response:
                response = response.split('Final Answer:')[-1].strip()
            
            # Split response into decision and reason
            parts = response.split(':', 1)
            if len(parts) == 2:
                decision, reason = parts[0].strip(), parts[1].strip()
                df_processed.at[idx, 'new_model_code'] = decision
                df_processed.at[idx, 'new_model_reason'] = reason
            else:
                df_processed.at[idx, 'new_model_code'] = None
                df_processed.at[idx, 'new_model_reason'] = f"Invalid format: {response}"
                
        except Exception as e:
            print(f"\nError processing row {idx}: {str(e)}")
            continue
    
    return df_processed

if __name__ == "__main__":
    # Process 100 tweets
    df_results = process_tweets_sample(df, n_rows=100)
    print("\nProcessing complete. Results stored in df_results")

  0%|          | 0/100 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
 I need to analyze the tweet using the TweetCodingTool.
Action: TweetCodingTool
Action Input: {"text": "A great guy (with great ratings)! https://t.co/S0S6xqrjSC", "code_name": "HSTG", "code_definition": "Are hashtags used in this tweet? Exclude hyperlinks from this decision."}

  1%|          | 1/100 [00:06<11:16,  6.83s/it]


Observation: Error:Invalid response format
Thought:

> Finished chain.


> Entering new AgentExecutor chain...
 I need to analyze the tweet using the TweetCodingTool.
Action: TweetCodingTool
Action Input: {"text": "A great guy (with great ratings)! https://t.co/S0S6xqrjSC", "code_name": "ATSN", "code_definition": "Are at signs (\"@\") used in this tweet? Include \"@\" that are present in retweets. Exclude hyperlinks from this decision."}

  2%|▏         | 2/100 [00:13<10:45,  6.58s/it]

...


Observation: Error:Invalid response format
Thought:

> Finished chain.


> Entering new AgentExecutor chain...
 I need to use the TweetCodingTool to analyze the tweet.
Action: TweetCodingTool
Action Input: {"text": "The U.S. Coast Guard FEMA and all Federal and State brave people are ready. Here comes Irma. God bless everyone!", "code_name": "CAPT", "code_definition": "Are there words that contain only capital letters in this tweet?"}

 99%|█████████▉| 99/100 [10:47<00:07,  7.83s/it]


Observation: Error:Invalid response format
Thought:

> Finished chain.


> Entering new AgentExecutor chain...
 I need to use the TweetCodingTool to analyze the tweet.
Action: TweetCodingTool
Action Input: {"text": "The U.S. Coast Guard FEMA and all Federal and State brave people are ready. Here comes Irma. God bless everyone!", "code_name": "INDV", "code_definition": "Does this tweet reference an individual person? Exclude self-references to Donald Trump."}

100%|██████████| 100/100 [10:55<00:00,  6.56s/it]


Observation: Error:Invalid response format
Thought:

> Finished chain.

Processing complete. Results stored in df_results


In [39]:
import os
from dotenv import load_dotenv
import pandas as pd
from tqdm import tqdm
import requests
import json

# Load environment variables
load_dotenv()

def prepare_prompt(text, code_name, code_definition):
    """Prepare prompt for a specific code and text."""
    return f"""<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>

<|user|>Analyze this tweet using the following code definition:

TWEET: {text}

CODE: {code_name}
DEFINITION: {code_definition}

Respond with exactly "Yes:" or "No:" followed by your reason.</|user|>

<|assistant|>Let me analyze this tweet according to the definition.
"""

def analyze_tweet(text, code_name, code_definition):
    """Analyze a tweet using Together API with detailed debugging."""
    headers = {
        "Authorization": f"Bearer {os.getenv('TOGETHER_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    prompt = prepare_prompt(text, code_name, code_definition)
    data = {
        "model": "meta-llama/Llama-3-70b-chat-hf",
        "prompt": prompt,
        "max_tokens": 150,
        "temperature": 0.1,
        "top_p": 0.9,
        "top_k": 50,
        "repetition_penalty": 1.1
    }
    
    try:
        print("\nDebug - Sending request to API...")
        print(f"Debug - Prompt:\n{prompt}")
        
        response = requests.post(
            "https://api.together.xyz/inference",
            headers=headers,
            json=data
        )
        
        print(f"\nDebug - API Status Code: {response.status_code}")
        response_json = response.json()
        print(f"Debug - Full API Response: {json.dumps(response_json, indent=2)}")
        
        if response.status_code == 200:
            response_text = response_json.get('output', {}).get('choices', [{}])[0].get('text', '')
            print(f"Debug - Extracted response text: {response_text}")
            return response_text.strip()
        else:
            print(f"Debug - API Error: {response.text}")
            return f"Error: API returned status {response.status_code}"
        
    except Exception as e:
        print(f"\nDebug - Exception: {str(e)}")
        return f"Error: {str(e)}"

def process_tweets_sample(df, codebook, n_rows=2):
    """Process tweets with detailed debugging output."""
    df_processed = df.copy()
    df_processed['new_model_code'] = None
    df_processed['new_model_reason'] = None
    
    for idx, row in tqdm(df.iloc[:n_rows].iterrows(), total=n_rows):
        try:
            print(f"\n{'='*50}")
            print(f"Processing tweet {idx}")
            print(f"Text: {row['text']}")
            print(f"Code: {row['code_name']}")
            
            # Get response
            response = analyze_tweet(
                row['text'],
                row['code_name'],
                codebook[row['code_name']]
            )
            
            print(f"\nDebug - Response from analyze_tweet: {response}")
            
            # Parse response
            if ':' in response:
                decision, reason = response.split(':', 1)
                decision = decision.strip()
                reason = reason.strip()
                
                if decision.lower() in ['yes', 'no']:
                    df_processed.at[idx, 'new_model_code'] = decision.title()
                    df_processed.at[idx, 'new_model_reason'] = reason
                    print(f"Success - Decision: {decision}, Reason: {reason}")
                else:
                    print(f"Invalid decision: {decision}")
            else:
                print(f"Invalid response format: {response}")
            
        except Exception as e:
            print(f"Error processing row {idx}: {str(e)}")
            continue
    
    return df_processed

# Your codebook
codebook = {
    'HSTG': "Are hashtags used in this tweet? Exclude hyperlinks from this decision.",
    'ATSN': "Are at signs (\"@\") used in this tweet? Include \"@\" that are present in retweets. Exclude hyperlinks from this decision.",
    'CRIT': "Does Donald Trump criticize another person or idea in this tweet? If the author suggests at any point in the tweet that some person or entity did something wrong, code 'Yes'.",
    'MEDI': "Does Donald Trump make derogatory or condescending statements about the news media in this tweet?",
    'FAMY': "Does Donald Trump reference members of his immediate family in this tweet?",
    'PLCE': "Does this tweet reference the police?",
    'MAGA': "Does this tweet reference Donald Trump's 2016 campaign slogan?",
    'CAPT': "Are there words that contain only capital letters in this tweet?",
    'INDV': "Does this tweet reference an individual person? Exclude self-references to Donald Trump.",
    'MARG': "Does the tweet explicitly reference a marginalized group or groups?",
    'INTN': "Does this tweet reference international topics outside of USA?",
    'PRTY': "Does this tweet reference US political parties?",
    'IMMG': "Does this tweet reference immigration in the US?"
}

if __name__ == "__main__":
    # Try with 100 tweets for more comprehensive testing
    print("\nTesting with 100 tweets...")
    df_test = process_tweets_sample(df, codebook, n_rows=100)
    print("\nTest results:")
    print(df_test[['text', 'code_name', 'new_model_code', 'new_model_reason']].head())


Testing with 100 tweets...


  0%|          | 0/100 [00:00<?, ?it/s]


Processing tweet 0
Text: A great guy (with great ratings)! https://t.co/S0S6xqrjSC
Code: HSTG

Debug - Sending request to API...
Debug - Prompt:
<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>

<|user|>Analyze this tweet using the following code definition:

TWEET: A great guy (with great ratings)! https://t.co/S0S6xqrjSC

CODE: HSTG
DEFINITION: Are hashtags used in this tweet? Exclude hyperlinks from this decision.

Respond with exactly "Yes:" or "No:" followed by your reason.</|user|>

<|assistant|>Let me analyze this tweet according to the definition.



  1%|          | 1/100 [00:00<01:37,  1.01it/s]


Debug - API Status Code: 200
Debug - Full API Response: {
  "id": "8edabb4b5b41abcf-CPH",
  "created_at": "2024-12-06T07:58:34.877Z",
  "status": "finished",
  "prompt": [
    "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n<|user|>Analyze this tweet using the following code definition:\n\nTWEET: A great guy (with great ratings)! https://t.co/S0S6xqrjSC\n\nCODE: HSTG\nDEFINITION: Are hashtags used in this tweet? Exclude hyperlinks from this decision.\n\nRespond with exactly \"Yes:\" or \"No:\" followed by your reason.</|user|>\n\n<|assistant|>Let me analyze this tweet according to the definition.\n"
  ],
  "model": "meta-llama/Llama-3-70b-chat-hf",
  "model_routed": "meta-llama/Llama-3-70b-chat-hf",
  "model_owner": "",
  "num_returns": 1,
  "args": {
    "model": "meta-llama/Llama-3-70b-chat-hf",
    "prompt": "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n<|user|>Analyze this tweet using the following c

...


Debug - API Status Code: 200
Debug - Full API Response: {
  "id": "8edabe1939d7be3f-CPH",
  "created_at": "2024-12-06T08:00:29.732Z",
  "status": "finished",
  "prompt": [
    "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n<|user|>Analyze this tweet using the following code definition:\n\nTWEET: The U.S. Coast Guard FEMA and all Federal and State brave people are ready. Here comes Irma. God bless everyone!\n\nCODE: MAGA\nDEFINITION: Does this tweet reference Donald Trump's 2016 campaign slogan?\n\nRespond with exactly \"Yes:\" or \"No:\" followed by your reason.</|user|>\n\n<|assistant|>Let me analyze this tweet according to the definition.\n"
  ],
  "model": "meta-llama/Llama-3-70b-chat-hf",
  "model_routed": "meta-llama/Llama-3-70b-chat-hf",
  "model_owner": "",
  "num_returns": 1,
  "args": {
    "model": "meta-llama/Llama-3-70b-chat-hf",
    "prompt": "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n<|

 99%|█████████▉| 99/100 [01:56<00:00,  1.10it/s]


Debug - API Status Code: 200
Debug - Full API Response: {
  "id": "8edabe1f7e7492e2-CPH",
  "created_at": "2024-12-06T08:00:30.741Z",
  "status": "finished",
  "prompt": [
    "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n<|user|>Analyze this tweet using the following code definition:\n\nTWEET: The U.S. Coast Guard FEMA and all Federal and State brave people are ready. Here comes Irma. God bless everyone!\n\nCODE: CAPT\nDEFINITION: Are there words that contain only capital letters in this tweet?\n\nRespond with exactly \"Yes:\" or \"No:\" followed by your reason.</|user|>\n\n<|assistant|>Let me analyze this tweet according to the definition.\n"
  ],
  "model": "meta-llama/Llama-3-70b-chat-hf",
  "model_routed": "meta-llama/Llama-3-70b-chat-hf",
  "model_owner": "",
  "num_returns": 1,
  "args": {
    "model": "meta-llama/Llama-3-70b-chat-hf",
    "prompt": "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n

100%|██████████| 100/100 [01:58<00:00,  1.18s/it]


Debug - API Status Code: 200
Debug - Full API Response: {
  "id": "8edabe24cfac8f63-CPH",
  "created_at": "2024-12-06T08:00:31.589Z",
  "status": "finished",
  "prompt": [
    "<|system|>You are a qualitative coder analyzing tweets using a codebook.</|system|>\n\n<|user|>Analyze this tweet using the following code definition:\n\nTWEET: The U.S. Coast Guard FEMA and all Federal and State brave people are ready. Here comes Irma. God bless everyone!\n\nCODE: INDV\nDEFINITION: Does this tweet reference an individual person? Exclude self-references to Donald Trump.\n\nRespond with exactly \"Yes:\" or \"No:\" followed by your reason.</|user|>\n\n<|assistant|>Let me analyze this tweet according to the definition.\n"
  ],
  "model": "meta-llama/Llama-3-70b-chat-hf",
  "model_routed": "meta-llama/Llama-3-70b-chat-hf",
  "model_owner": "",
  "num_returns": 1,
  "args": {
    "model": "meta-llama/Llama-3-70b-chat-hf",
    "prompt": "<|system|>You are a qualitative coder analyzing tweets using a 

In [8]:
df2 = pd.read_csv('tweet_analysis_final.csv')

In [10]:
df2

,text_id,code_id,code_name,text,original_code,replicated_code,model_code,reason,new_model_code,new_model_reason
0,1,1,HSTG,A great guy (with great ratings)! https://t.co...,No,No,No,"There are no hashtags used in this tweet, only...",NaN,Could not parse response:
1,1,2,ATSN,A great guy (with great ratings)! https://t.co...,No,No,No,There are no at signs (@) present in this twee...,NaN,Could not parse response:
2,1,3,CRIT,A great guy (with great ratings)! https://t.co...,No,No,No,The tweet does not criticize another person or...,NaN,Could not parse response:
3,1,4,MEDI,A great guy (with great ratings)! https://t.co...,No,No,No,This tweet does not contain any derogatory or ...,NaN,Could not parse response:
4,1,5,FAMY,A great guy (with great ratings)! https://t.co...,No,No,No,The tweet does not reference any members of Do...,NaN,Could not parse response:
...,...,...,...,...,...,...,...,...,...,...
1295,100,9,INDV,RT @realDonaldTrump: National Pearl Harbor Rem...,No,No,No,The tweet does not reference any individual pe...,NaN,NaN
1296,100,10,MARG,RT @realDonaldTrump: National Pearl Harbor Rem...,No,No,No,The tweet does not reference any marginalized ...,NaN,NaN
1297,100,11,INTN,RT @realDonaldTrump: National Pearl Harbor Rem...,No,No,No,The tweet does not reference any international...,NaN,NaN
1298,100,12,PRTY,RT @realDonaldTrump: National Pearl Harbor Rem...,No,No,No,This tweet does not reference any US political...,NaN,NaN


In [11]:
df2.new_model_code.value_counts()

Series([], Name: count, dtype: int64)